## Observations and Insights 

In [171]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Drop duplicates from data
study_data = study_results.drop_duplicates(subset=["Mouse ID", "Timepoint"], keep="first")

# Combine the data into a single dataset
cleaned_data = pd.merge(mouse_metadata,study_data, on="Mouse ID", how="inner")

# Display the data table for preview
cleaned_data

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1883,z969,Naftisol,Male,9,30,25,63.145652,2
1884,z969,Naftisol,Male,9,30,30,65.841013,3
1885,z969,Naftisol,Male,9,30,35,69.176246,4
1886,z969,Naftisol,Male,9,30,40,70.314904,4


In [172]:
# Checking the number of mice.
total_mice = cleaned_data["Mouse ID"].nunique()

print(f"In the study a total number of {total_mice} mice were used.")

In the study a total number of 249 mice were used.


In [207]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 

duplicate = pd.DataFrame(study_results.duplicated(subset=["Mouse ID", "Timepoint"]))

duplicate = duplicate.loc[duplicate[0] == True]

duplicate = pd.DataFrame(duplicate.index).rename(columns={0:"ID Number"})

duplicate

,ID Number
0,137
1,360
2,681
3,869
4,1111


In [200]:
# Optional: Get all the data for the duplicate mouse ID. 

difference = pd.DataFrame([study_results.iloc[duplicate[x],:] for x in range(len(duplicate))])

difference

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites
137,g989,0,45.000000,0
360,g989,5,47.570392,0
681,g989,10,49.880528,0
869,g989,15,53.442020,0
1111,g989,20,54.657650,1


In [175]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.


In [176]:
# Checking the number of mice in the clean DataFrame.


## Summary Statistics

In [177]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# This method is the most straighforward, creating multiple series and putting them all together at the end.



In [178]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# This method produces everything in a single groupby function


## Bar and Pie Charts

In [179]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pandas. 



In [180]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pyplot.



In [181]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [182]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [183]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [184]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [185]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [186]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin


In [187]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen


## Correlation and Regression

In [188]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
